In [2]:
import requests 
from bs4 import BeautifulSoup
import json
import csv

In [3]:
class ZillowScraper:
    results = []
    headers = {
        'accept': '*/*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,ru;q=0.8',
        'cookie': 'zguid=23|%240bc9d364-2ef6-4c88-a0d3-863ec03ebaf7; zgsession=1|0193e537-a954-42fe-983e-3bb4cc763f49; G_ENABLED_IDPS=google; JSESSIONID=1D4343D337FD7405C27D44070B86F926; AWSALB=2DdHKCsF6rJtIVTkfkgMPaz3OJwMCjUFqr9QNCSWUqqGe82VQtLsdaudWa3qmvwn842yku47FKgvv/6T6WaqWT0/zXZVQMGv+hOitv3L1axs9N/NE5PTXBmEVRxV; AWSALBCORS=2DdHKCsF6rJtIVTkfkgMPaz3OJwMCjUFqr9QNCSWUqqGe82VQtLsdaudWa3qmvwn842yku47FKgvv/6T6WaqWT0/zXZVQMGv+hOitv3L1axs9N/NE5PTXBmEVRxV; search=6|1589475984405%7Crect%3D40.915266%252C-73.765273%252C40.785743%252C-73.933405%26rid%3D17182%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26sort%3Ddays%26z%3D1%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%09%0917182%09%09%09%09%09%09',
        'referer': 'https://www.zillow.com/bronx-new-york-ny/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Bronx%2C%20New%20York%2C%20NY%22%2C%22mapBounds%22%3A%7B%22west%22%3A-73.933405%2C%22east%22%3A-73.765273%2C%22south%22%3A40.785743%2C%22north%22%3A40.915266%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A17182%2C%22regionType%22%3A17%7D%5D%2C%22filterState%22%3A%7B%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%7D%7D',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (iPad; CPU OS 11_0 like Mac OS X) AppleWebKit/604.1.34 (KHTML, like Gecko) Version/11.0 Mobile/15A5341f Safari/604.1',
    }
   
    def fetch(self, url,params):
        print('HTTP GET request to URL: %s'%url, end='')
        res = requests.get(url , params = params, headers = self.headers)
        print(' | Status code: %s' % res.status_code)
        
        return res
        
    def save_response(self, res):
        with open('res.html', 'w',encoding='utf-8') as html_file:
            html_file.write(res)
    
    def load_response(self):
        html = ''
        
        with open('res.html','r',encoding='utf-8') as html_file:
            for line in html_file:
                html+=line
        return html
    
    def parse(self,html):
        content = BeautifulSoup(html, 'lxml')
        cards = content.findAll('article',{'class':'list-card'})
            
        for card in cards:
            try:
                squareFeet = card.find('ul',{'class':'list-card-details'}).findAll('li')[2].text.split(' ')[0]
            except: 
                squareFeet = 'N/A'
                
            if(card.find('ul',{'class':'list-card-details'}).find('li').text.startswith( '$' )):
                price = card.find('ul',{'class':'list-card-details'}).find('li').text.split(' ')[0].replace("+", "")
                bathrooms = "1"
                squareFeet = 'N/A'
                try:
                    bedroom = card.find('ul',{'class':'list-card-details'}).find('li').text.split('+ ')[1].split(' ')[0]
                except:
                    bedroom = card.find('ul',{'class':'list-card-details'}).find('li').text.split(' ')[1].split(' ')[0]
            else:
                price = card.find('div',{'class':'list-card-price'}).text.split('/')[0]
                bedroom = card.find('ul',{'class':'list-card-details'}).find('li').text.split(' ')[0]
                bathrooms = card.find('ul',{'class':'list-card-details'}).findAll('li')[1].text.split(' ')[0]
                try:
                    squareFeet = card.find('ul',{'class':'list-card-details'}).findAll('li')[2].text.split(' ')[0]
                except: 
                    squareFeet = 'N/A'
                
            if ('| ') in card.find('address',{'class':'list-card-addr'}).text:
                address = card.find('address',{'class':'list-card-addr'}).text.split('| ')[1]
            else:
                address = card.find('address',{'class':'list-card-addr'}).text
                
            self.results.append({
                'price': price,
                'address':address,
                'bedrooms':bedroom,
                'bathrooms':bathrooms,
                 'squareFeet':squareFeet
            })
    def to_csv(self):
        with open('zillowBronx.csv','w') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=self.results[0].keys(),lineterminator = '\n')
            writer.writeheader()
            for row in self.results:
                writer.writerow(row)

    def run(self):
        firstPage = True
        pPage = 2
        pageNum = 3
        for page in range(1,15):
            params = {
                'searchQueryState' : '{"pagination":{"currentPage":%s},"usersSearchTerm":"Bronx, New York, NY","mapBounds":{"west":-73.933405,"east":-73.765273,"south":40.785743,"north":40.915266},"regionSelection":[{"regionId":17182,"regionType":17}],"filterState":{"ah":{"value":true},"fr":{"value":true},"fsba":{"value":false},"fsbo":{"value":false},"nc":{"value":false},"cmsn":{"value":false},"auc":{"value":false},"fore":{"value":false},"pmf":{"value":false},"pf":{"value":false}}}'%page 
                }
            res = self.fetch('https://www.zillow.com/bronx-new-york-ny/rentals/',params)
            self.parse(res.text)
            self.to_csv()
#                 firstPage = False
#             else:
#                 params = {
#                     'searchQueryState': '{"pagination":{"currentPage":'+str(pageNum)+',"usersSearchTerm":"Brooklyn, New York, NY","mapBounds":{"west":-74.26086287280273,"east":-73.61438612719726,"south":40.50200122440905,"north":40.80804134081856},"mapZoom":12,"regionSelection":[{"regionId":37607,"regionType":17}],"filterState":{"pmf":{"value":false},"fore":{"value":false},"auc":{"value":false},"nc":{"value":false},"fr":{"value":true},"fsbo":{"value":false},"cmsn":{"value":false},"pf":{"value":false},"fsba":{"value":false}},"isListVisible":true,"isMapVisible":false}'
#                          }
#                 res = self.fetch('https://www.zillow.com/brooklyn-new-york-ny/rentals/'+str(pageNum)+'_p/',params)
#                 self.parse(res.text)
#                 self.to_csv()
#                 pageNum = int(pageNum) + 1
#                 pPage = int(pPage) + 1
        
if __name__ == '__main__':
    
    scraper = ZillowScraper()
    scraper.run()

HTTP GET request to URL: https://www.zillow.com/bronx-new-york-ny/rentals/

 | Status code: 200
HTTP GET request to URL: https://www.zillow.com/bronx-new-york-ny/rentals/

 | Status code: 200
HTTP GET request to URL: https://www.zillow.com/bronx-new-york-ny/rentals/

 | Status code: 200
HTTP GET request to URL: https://www.zillow.com/bronx-new-york-ny/rentals/

 | Status code: 200


...

 | Status code: 200
HTTP GET request to URL: https://www.zillow.com/bronx-new-york-ny/rentals/

 | Status code: 200
HTTP GET request to URL: https://www.zillow.com/bronx-new-york-ny/rentals/

 | Status code: 200
HTTP GET request to URL: https://www.zillow.com/bronx-new-york-ny/rentals/

 | Status code: 200
HTTP GET request to URL: https://www.zillow.com/bronx-new-york-ny/rentals/

 | Status code: 200
